In [93]:
import pandas as pd
import numpy as np

import seaborn as sns

from dask import dataframe as dd
import dask

import pickle

from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.optimizers import SGD

import tensorflow as tf

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.model_selection import RepeatedKFold, cross_val_score, train_test_split

from dask_ml.linear_model import LogisticRegression
from dask_ml.preprocessing import OneHotEncoder, MinMaxScaler
from dask_ml.model_selection import train_test_split

In [94]:
not_enough_ram = False

## Загрузка датафреймов

In [95]:
# df = pd.read_pickle('data/ga_sessions.pkl')
# print(df.shape)
# df.head()

In [96]:
# df_hits = pd.read_pickle('data/ga_hits.pkl')
# print(df_hits.shape)
# df_hits.head()

In [97]:
# df['target'] = df['session_id'].isin(df_hits['session_id'].unique()).apply(lambda x: 1 if x else 0)
# df.head()

In [98]:
# df.to_pickle('data/data_with_target.pickle')

In [99]:
df = pd.read_pickle('data/data_with_target.pickle')

## Data cleaning

### Заполнение пустых строк

In [100]:
df.isna().sum() * 100 / len(df)

session_id                   0.000000
client_id                    0.000000
visit_date                   0.000000
visit_time                   0.000000
visit_number                 0.000000
utm_source                   0.005215
utm_medium                   0.000000
utm_campaign                11.806346
utm_adcontent               18.043410
utm_keyword                 58.174009
device_category              0.000000
device_os                   57.533002
device_brand                 6.380394
device_model                99.121633
device_screen_resolution     0.000000
device_browser               0.000000
geo_country                  0.000000
geo_city                     0.000000
target                       0.000000
dtype: float64

In [101]:
df = df.drop(['device_model', 'utm_keyword'], axis=1)

In [102]:
df.device_os.value_counts(dropna=False)

None             1070138
Android           464054
iOS               207104
Windows            88307
Macintosh          24824
Linux               5120
(not set)            364
Chrome OS             83
BlackBerry            27
Tizen                  7
Samsung                4
Windows Phone          4
Firefox OS             3
Nokia                  3
Name: device_os, dtype: int64

In [103]:
df['device_os'] = df['device_os'].fillna('other')

In [104]:
df['device_brand'].value_counts(dropna=False)

Apple       551088
Samsung     332194
Xiaomi      288367
            248500
Huawei      185853
             ...  
Vodafone         1
Wexler           1
KingSing         1
Star             1
Opera            1
Name: device_brand, Length: 207, dtype: int64

In [105]:
df['device_brand'] = df['device_brand'].fillna('other')

In [106]:
df['utm_campaign'].mode()

0    LTuZkdKfxRGVceoWkVyg
Name: utm_campaign, dtype: object

In [107]:
df['utm_source'].mode()

0    ZpYIoDJMcFzVoPFsHGJL
Name: utm_source, dtype: object

In [108]:
df['utm_adcontent'].mode()

0    JNHcPlZPxEMWDnRiyoBf
Name: utm_adcontent, dtype: object

In [109]:
df['utm_campaign'] = df['utm_campaign'].fillna(df['utm_campaign'].mode())
df['utm_source'] = df['utm_source'].fillna(df['utm_source'].mode())
df['utm_adcontent'] = df['utm_adcontent'].fillna(df['utm_adcontent'].mode())

### Приведение типов данных

In [110]:
df.dtypes

session_id                  object
client_id                   object
visit_date                  object
visit_time                  object
visit_number                 int64
utm_source                  object
utm_medium                  object
utm_campaign                object
utm_adcontent               object
device_category             object
device_os                   object
device_brand                object
device_screen_resolution    object
device_browser              object
geo_country                 object
geo_city                    object
target                       int64
dtype: object

In [111]:
df.head()

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,device_category,device_os,device_brand,device_screen_resolution,device_browser,geo_country,geo_city,target
0,9055434745589932991.1637753792.1637753792,2108382700.1637753791,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,mobile,Android,Huawei,360x720,Chrome,Russia,Zlatoust,1
1,905544597018549464.1636867290.1636867290,210838531.1636867288,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,mobile,Android,Samsung,385x854,Samsung Internet,Russia,Moscow,1
2,9055446045651783499.1640648526.1640648526,2108385331.1640648523,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,mobile,Android,Huawei,360x720,Chrome,Russia,Krasnoyarsk,1
3,9055447046360770272.1622255328.1622255328,2108385564.1622255328,2021-05-29,05:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,NaN,NOBKLgtuvqYWkXQHeYWM,mobile,other,Xiaomi,393x786,Chrome,Russia,Moscow,1
4,9055447046360770272.1622255345.1622255345,2108385564.1622255328,2021-05-29,05:00:00,2,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,mobile,other,Xiaomi,393x786,Chrome,Russia,Moscow,1


In [112]:
df['visit_date'] = pd.to_datetime(df['visit_date'])

### Обработка выбросов

In [113]:
def calculate_outliers(data):
    q25 = data.quantile(0.25)
    q75 = data.quantile(0.75)
    iqr = q75 - q25
    boundaries = (q25 - 1.5 * iqr, q75 + 1.5 * iqr)
    
    return boundaries

In [114]:
df.visit_number.describe()

count    1.860042e+06
mean     2.712804e+00
std      1.182907e+01
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      2.000000e+00
max      5.640000e+02
Name: visit_number, dtype: float64

In [115]:
boundaries = calculate_outliers(df['visit_number'])
b_max = boundaries[1]
round(b_max)

4

In [116]:
df.loc[df['visit_number'] > b_max, 'visit_number'] = round(b_max)

In [117]:
df.visit_number.describe()

count    1.860042e+06
mean     1.483161e+00
std      9.356721e-01
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      2.000000e+00
max      4.000000e+00
Name: visit_number, dtype: float64

### Объединение редких значений

In [118]:
df.geo_city.describe()

count     1860042
unique       2548
top        Moscow
freq       805329
Name: geo_city, dtype: object

In [119]:
geo_city_vc = df.geo_city.value_counts()

In [120]:
unpopular_city = [k for k, v in geo_city_vc.items() if v <= geo_city_vc.mean()]

In [121]:
pd.DataFrame(unpopular_city).to_csv('data/unpopular_city.csv', index=False)

In [122]:
del geo_city_vc

In [123]:
df.loc[df['geo_city'].isin(unpopular_city), 'geo_city'] = 'other'

In [124]:
del unpopular_city

## Feature engineering

In [125]:
df.describe(include='all', datetime_is_numeric=True)

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,device_category,device_os,device_brand,device_screen_resolution,device_browser,geo_country,geo_city,target
count,1860042,1860042,1860042,1860042,1.860042e+06,1859945,1860042,1640439,1524427,1860042,1860042,1860042,1860042,1860042,1860042,1860042,1.860042e+06
unique,1860042,1391719,NaN,85318,NaN,293,56,412,286,3,14,207,5039,57,166,145,NaN
top,9055434745589932991.1637753792.1637753792,1750498477.1629450062,NaN,12:00:00,NaN,ZpYIoDJMcFzVoPFsHGJL,banner,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,mobile,other,Apple,414x896,Chrome,Russia,Moscow,NaN
freq,1,462,NaN,61067,NaN,578290,552272,463481,1006599,1474871,1070138,551088,169090,1013436,1800565,805329,NaN
mean,NaN,NaN,2021-09-26 11:45:55.389394176,NaN,1.483161e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.313048e-01
min,NaN,NaN,2021-05-19 00:00:00,NaN,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00
25%,NaN,NaN,2021-08-02 00:00:00,NaN,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+00
50%,NaN,NaN,2021-10-06 00:00:00,NaN,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+00
75%,NaN,NaN,2021-11-23 00:00:00,NaN,2.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+00
max,NaN,NaN,2021-12-31 00:00:00,NaN,4.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+00


### Органический трафик

In [126]:
df.utm_medium.value_counts(dropna=False)

banner               552272
cpc                  434794
(none)               300575
cpm                  242083
referral             152050
organic               63034
email                 29240
push                  28035
stories               10582
cpv                    8022
blogger_channel        8015
smartbanner            6794
blogger_stories        4312
cpa                    4279
tg                     4011
app                    2836
post                   2326
smm                    1985
outlook                1332
clicks                  934
blogger_header          771
(not set)               480
info_text               343
sms                     239
landing                 134
partner                  97
fb_smm                   66
vk_smm                   65
link                     57
cbaafe                   47
CPM                      40
yandex_cpc               36
ok_smm                   28
static                   22
google_cpc               20
article             

In [127]:
df['is_organic'] = df.utm_medium.isin(['organic', 'referral', '(none)']).apply(lambda x: 1 if x else 0)

In [128]:
df['is_organic'].value_counts(dropna=False)

0    1344383
1     515659
Name: is_organic, dtype: int64

### Реклама в социальных сетях

In [129]:
advertising_tags = ['QxAxdyPLuQMEcrdZWdWb', 'MvfHsxITijuriZxsqZqt', 'ISrKoXQCxqqYvAZICvjs',
                    'IZEXUFLARCUMynmHNBGo', 'PlbkrSYoHuZBWfYjYnfw',
                    'gVRrcxiDQubJiljoTbGm']

df['is_advertising'] = df['utm_source'].isin(advertising_tags).apply(lambda x: 1 if x else 0)

In [130]:
df['is_advertising'].value_counts()

0    1585815
1     274227
Name: is_advertising, dtype: int64

### День недели

In [131]:
df['dayofweek'] = df['visit_date'].dt.dayofweek.astype('str')

In [132]:
df['dayofweek'].value_counts()

1    305130
0    300906
3    274462
4    268819
2    258616
5    226397
6    225712
Name: dayofweek, dtype: int64

### Час посещения 

In [133]:
df['visit_time_hour'] = df['visit_time'].apply(lambda x: str(x.hour))

In [134]:
df['visit_time_hour'].value_counts()

14    111487
16    105241
15    103854
18    102701
13    102477
12    102071
11    101266
17    101086
10    100849
19     99184
20     98731
21     97612
22     93473
9      91860
23     84643
8      72015
0      66202
7      54894
1      40080
6      36440
2      27328
5      25292
3      21036
4      20220
Name: visit_time_hour, dtype: int64

### Разрешение экрана по осям

In [135]:
df['device_screen_resolution'].value_counts()

414x896      169090
1920x1080    125768
375x812      117944
393x851      115454
375x667       93341
              ...  
2262x1553         1
1097x617          1
421x847           1
1791x1007         1
464x1123          1
Name: device_screen_resolution, Length: 5039, dtype: int64

In [136]:
screen_mode = df['device_screen_resolution'].mode()[0]
screen_mode

'414x896'

In [137]:
df.loc[df['device_screen_resolution'] == '(not set)', 'device_screen_resolution'] = screen_mode
df['device_screen_resolution_x'] = df['device_screen_resolution'].apply(lambda x: int(x.split('x')[0]))
df['device_screen_resolution_y'] = df['device_screen_resolution'].apply(lambda x: int(x.split('x')[1]))

In [138]:
df['device_screen_resolution_x'].describe()

count    1.860042e+06
mean     6.567049e+02
std      5.531441e+02
min      0.000000e+00
25%      3.600000e+02
50%      3.930000e+02
75%      4.280000e+02
max      5.924000e+03
Name: device_screen_resolution_x, dtype: float64

In [139]:
df['device_screen_resolution_y'].describe()

count    1.860042e+06
mean     8.408741e+02
std      1.939633e+02
min      0.000000e+00
25%      7.600000e+02
50%      8.440000e+02
75%      8.960000e+02
max      2.000000e+04
Name: device_screen_resolution_y, dtype: float64

### Dask DF

In [140]:
categorical = ['utm_source', 'utm_medium', 'utm_campaign', 'utm_adcontent',
              'device_os', 'device_brand', 'device_browser',
              'geo_country', 'geo_city',
              'dayofweek', 'visit_time_hour']
numerical = ['visit_number', 'device_screen_resolution_x', 'device_screen_resolution_y']

In [141]:
ddf = dd.from_pandas(df[numerical], chunksize=20000)

### MinMax

In [142]:
%%time
mm_scaler = MinMaxScaler()
mm_scaler.fit(ddf[numerical])
pickle.dump(mm_scaler, open("data/mm_scaler.pickle", "wb"))

CPU times: user 82.7 ms, sys: 15.4 ms, total: 98.1 ms
Wall time: 83.9 ms


In [143]:
ddf[numerical] = mm_scaler.transform(ddf[numerical])

### OHE

In [144]:
ddf_categorical = dd.from_pandas(df[categorical], chunksize=20000).categorize()

In [145]:
ohe = OneHotEncoder(sparse=False)
ohe.fit(ddf_categorical)
pickle.dump(ohe, open("data/ohe.pickle", "wb"))
ohe_transform = ohe.transform(ddf_categorical)
del ddf_categorical

In [146]:
ddf = dd.merge(ddf, ohe_transform.astype('uint8'))

In [147]:
del ohe_transform

### Перенос целевой переменной

In [148]:
ddf['target'] = df['target']

In [149]:
# del df

In [150]:
df = ddf.compute()

In [151]:
df.shape

(1860042, 1671)

### Кореляция целевой переменной с другими атрибутами

In [152]:
corrs = df.corrwith(df["target"]).sort_values(key=lambda x: abs(x), ascending=False)

In [153]:
corrs

target                               1.000000e+00
utm_campaign_TmThBvoCcwkCZZUWACYq   -1.771009e-01
utm_source_SzZERoLMmrEUEhDaYcyN     -1.304266e-01
utm_campaign_LEoPHuyFvzoNfnzGgfcd    1.092896e-01
utm_campaign_SgIUDYUKnyWHVowUOqid   -1.059165e-01
                                         ...     
geo_city_Shchyolkovo                 1.832487e-05
geo_country_Estonia                 -7.000858e-06
geo_city_Oryol                       3.980800e-06
geo_city_Tashkent                    3.429793e-06
geo_country_Lithuania               -3.316269e-07
Length: 1671, dtype: float64

In [154]:
# important_columns = corrs[abs(corrs) >= 0.003].index
important_columns = corrs[corrs >= 0.001].index
important_columns

Index(['target', 'utm_campaign_LEoPHuyFvzoNfnzGgfcd',
       'utm_adcontent_vCIpmpaGBnIQhyYNkXqp', 'device_os_other',
       'utm_campaign_gecBYcKZCPMcVYdSSzKP', 'geo_country_Russia',
       'utm_source_ZpYIoDJMcFzVoPFsHGJL', 'utm_medium_banner', 'device_brand_',
       'utm_adcontent_PkybGvWbaqORmxjNunqZ',
       ...
       'geo_city_Vladimir', 'geo_city_Saratov', 'geo_city_Vsevolozhsk',
       'utm_adcontent_dlwYatgsQMvLsNApVgcw',
       'utm_adcontent_xLqUcIHZLQWTXrgoxpZY',
       'utm_campaign_DBLWVjTjVZuzWYWTaDfZ', 'utm_source_azajeHUvPOKkHBvWFbjz',
       'utm_adcontent_kZBJHrVaekBwnsMoBrtF', 'geo_city_Ulyanovsk',
       'geo_city_Yoshkar-Ola'],
      dtype='object', length=386)

In [155]:
pd.DataFrame(important_columns).to_csv('data/important_columns.csv')

### Удаление ненужных колонок

In [156]:
df = df[important_columns]

### Сохранение датафрейма

In [157]:
df.to_pickle('data/df_prep.pkl')

# Modeling

### Загрузка датафрейма

In [158]:
if not_enough_ram:
    chunksize = 100000
    ddf = dd.from_pandas(pd.read_pickle('data/df_prep.pkl'), chunksize=chunksize)
else:
    df = pd.read_pickle('data/df_prep.pkl')

### train test split

In [159]:
def part_train_test(ddf, target, train_size, test_size):
    part = ddf.partitions[0]
    train = part.head(train_size)
    test = part.tail(test_size)
    for p in range(1, ddf.npartitions):
        part = ddf.partitions[p]
        train = pd.concat([train, part.head(train_size)])
        test = pd.concat([test, part.tail(test_size)])

    return tf.convert_to_tensor(train.drop(target, axis=1)), \
           tf.convert_to_tensor(test.drop(target, axis=1)), \
           tf.convert_to_tensor(train[target]), \
           tf.convert_to_tensor(test[target])

In [160]:
def train_iterator(ddf, target, size):
    while True:
        for p in range(1, ddf.npartitions):
            X_train = ddf.partitions[p].drop(target, axis=1).head(size)
            y_train = ddf.partitions[p][target].head(size)
            yield tf.convert_to_tensor(X_train), tf.convert_to_tensor(y_train)

In [161]:
%%time
if not_enough_ram:
    X_train, X_test, y_train, y_test = part_train_test(ddf, 'target', 0, int(chunksize*0.2))
else:
    X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1),
                                                    df['target'],
                                                    test_size=0.3,
                                                    shuffle=True,
                                                    random_state=42)
    del df

CPU times: user 1.69 s, sys: 209 ms, total: 1.9 s
Wall time: 1.9 s


### Неудачные модели

In [162]:
# model = LogisticRegression(max_iter=10,
#                            multi_class='multinomial',
#                            solver='newton',
#                            penalty='none',
#                            solver_kwargs={"normalize":False},
#                            random_state=42)
#
# model = xgb.XGBClassifier(random_state=42,
#                           n_estimators=100,
#                           learning_rate=0.1,
#                           max_depth=5,
#                           eval_metric='error')

## Keras

In [163]:
def buildmodel():
    model = Sequential()
    model.add(Dense(X_train.shape[1], input_dim=X_train.shape[1], activation='gelu'))
    model.add(Dense(256, input_dim=X_train.shape[1], activation='gelu'))
    model.add(Dense(128, input_dim=256, activation='gelu'))
    model.add(Dense(64, input_dim=128, activation='gelu'))
    model.add(Dense(1, input_dim=64, activation='linear'))
    sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='mean_squared_error', optimizer=sgd)
    return model


# model = load_model('data/keras_seq.h5')
model = buildmodel()
# model = pickle.load(open('data/keras_seq.pickle', 'rb'))

In [164]:
if not_enough_ram:
    gen = train_iterator(ddf, 'target', int(chunksize*0.8))
    while True:
        # with tf.device('/gpu:0'):
        X_train, y_train = next(gen)
        model.fit(X_train,
                  y_train,
                  epochs=1,
                  verbose=1)
else:
    model.fit(X_train,
              y_train,
              epochs=1,
              verbose=1)

   11/40689 [..............................] - ETA: 3:35 - loss: 0.3481   

2022-08-16 16:38:44.800597: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


40689/40689 [==============================] - 184s 5ms/step - loss: 0.0583


In [165]:
# pickle.dump(model, open('data/keras_seq.pickle', 'wb'))
model.save('data/keras_seq.h5')

In [166]:
predicted_train = model.predict(X_train)

   98/40689 [..............................] - ETA: 1:03

2022-08-16 16:41:52.436563: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


40689/40689 [==============================] - 63s 2ms/step


In [167]:
roc_auc_score(y_train, predicted_train)

0.8019607076262991

In [168]:
predicted_test = model.predict(X_test)

17438/17438 [==============================] - 28s 2ms/step


In [169]:
roc_auc_score(y_test, predicted_test)

0.8038873310774572